In [448]:
!pip3 install faker

## IMPORTAR MÓDULOS NECESARIOS

In [449]:
import numpy as np
import pandas as pd
from faker import Factory
from hashlib import sha256, blake2b, algorithms_available

In [450]:
fake = Factory.create("es_ES")

In [451]:
data = []
for _ in range(100):
    age = fake.random_int(min=18, max=80)
    data.append({
        'Name': fake.name(),
        'Age': age,
        'Email': fake.email(),
        'Phone': fake.phone_number(),
        'Postal Code': fake.postcode(),
        'Credit Card': fake.credit_card_number()
    })

In [452]:
df = pd.DataFrame(data)
df.head()

,Name,Age,Email,Phone,Postal Code,Credit Card
0,Estefanía Vendrell-Blanch,47,benitezedmundo@example.com,+34979 52 98 66,02687,343650255790246
1,Jordi del Mascaró,70,zpaniagua@example.net,+34945 649 215,28629,6574611792982362
2,Ruy Teruel Fabra,49,xtrillo@example.com,+34 824 67 56 26,21062,346345801365331
3,Máxima Porcel Pinto,77,valverdejose-antonio@example.com,+34 946 027 028,20734,502042473467
4,Jenny del Guillen,42,soledadferrer@example.net,+34 877 512 059,43733,3550825351276181


## PROCESO DE ANONIMIZACIÓN

Dado que no he encontrado conjuntos de datos lo suficientemente completos o con los campos necesarios para llevar a cabo esta práctica, he decidido crear el conjunto de datos con la ayuda del módulo Faker. Una vez generado el conjunto de datos, procedo con la anonimización de los datos haciendo uso de los métodos necesarios para ello.

Definimos funciones para poder anonimizar los diferentes datos que aparacen en nuestro dataset.

In [453]:
def encodeSHA256(df, column):
  return df[column].apply(
    lambda x: sha256(x.encode('utf-8')).hexdigest()
  )

def generalizeStringCode(df, column, level):
  return df[column].apply(
    lambda x: x[:-level] + ('*' * level)
  )

def generalizeEmail(email):
    username, domain = email.split('@')
    anonymized_username = username[:-(len(username)-3)] + '*' * (len(username)-3)
    return anonymized_username + '@' + domain

Al campo correspondiente a la tarjeta de crédito le aplicamos un Hash.

In [454]:
df["Credit Card"] = encodeSHA256(df, "Credit Card")
df.head()

,Name,Age,Email,Phone,Postal Code,Credit Card
0,Estefanía Vendrell-Blanch,47,benitezedmundo@example.com,+34979 52 98 66,02687,5d9f454ce695dc7953f7423f5d997f8f3b91a6c3b0c096...
1,Jordi del Mascaró,70,zpaniagua@example.net,+34945 649 215,28629,ead1a4eb7f082eb2e60f523cbcee0dca2a79ce377d4612...
2,Ruy Teruel Fabra,49,xtrillo@example.com,+34 824 67 56 26,21062,02c972f1f91748f7e6bc95f2572b9bd7a0375bfa26e6d6...
3,Máxima Porcel Pinto,77,valverdejose-antonio@example.com,+34 946 027 028,20734,403ee98f2e0ed94d63e37d2220925c19bbe0a3c4d48ca4...
4,Jenny del Guillen,42,soledadferrer@example.net,+34 877 512 059,43733,5ea8829400ca4995fb7d9d951a206633bb0c42a24b5833...


Para la edad estableceremos rangos donde el proceso de localizarla se hará más dificil.

In [455]:
rangos_edad = {
    'junior': {'min': 0, 'max': 30},
    'medium': {'min': 31, 'max': 50},
    'senior': {'min': 51, 'max': 100}
}
df["Age"] = df["Age"].apply(lambda x: next((key for key, value in rangos_edad.items() if value['min'] <= x <= value['max']), 'NA'))
df.head()


,Name,Age,Email,Phone,Postal Code,Credit Card
0,Estefanía Vendrell-Blanch,medium,benitezedmundo@example.com,+34979 52 98 66,02687,5d9f454ce695dc7953f7423f5d997f8f3b91a6c3b0c096...
1,Jordi del Mascaró,senior,zpaniagua@example.net,+34945 649 215,28629,ead1a4eb7f082eb2e60f523cbcee0dca2a79ce377d4612...
2,Ruy Teruel Fabra,medium,xtrillo@example.com,+34 824 67 56 26,21062,02c972f1f91748f7e6bc95f2572b9bd7a0375bfa26e6d6...
3,Máxima Porcel Pinto,senior,valverdejose-antonio@example.com,+34 946 027 028,20734,403ee98f2e0ed94d63e37d2220925c19bbe0a3c4d48ca4...
4,Jenny del Guillen,medium,soledadferrer@example.net,+34 877 512 059,43733,5ea8829400ca4995fb7d9d951a206633bb0c42a24b5833...


Para el código postal, en primer lugar aplicaremos ruido. Una vez completado este paso, procederemos a definir rangos y ubicaremos los códigos postales de cada usuario dentro de dichos rangos.

In [456]:
df["Postal Code"] = generalizeStringCode(df, "Postal Code", 3)
df.head()

,Name,Age,Email,Phone,Postal Code,Credit Card
0,Estefanía Vendrell-Blanch,medium,benitezedmundo@example.com,+34979 52 98 66,02***,5d9f454ce695dc7953f7423f5d997f8f3b91a6c3b0c096...
1,Jordi del Mascaró,senior,zpaniagua@example.net,+34945 649 215,28***,ead1a4eb7f082eb2e60f523cbcee0dca2a79ce377d4612...
2,Ruy Teruel Fabra,medium,xtrillo@example.com,+34 824 67 56 26,21***,02c972f1f91748f7e6bc95f2572b9bd7a0375bfa26e6d6...
3,Máxima Porcel Pinto,senior,valverdejose-antonio@example.com,+34 946 027 028,20***,403ee98f2e0ed94d63e37d2220925c19bbe0a3c4d48ca4...
4,Jenny del Guillen,medium,soledadferrer@example.net,+34 877 512 059,43***,5ea8829400ca4995fb7d9d951a206633bb0c42a24b5833...


In [457]:
valores = df["Postal Code"].unique()
print(f"Mínimo: {min(valores)}\nMáximo: {max(valores)}")

Mínimo: 01***
Máximo: 51***


In [458]:
rango_codigo_postal = {
    '01***-10***': {'min': '01***', 'max': '10***'},
    '11***-20***': {'min': '11***', 'max': '20***'},
    '21***-30***': {'min': '21***', 'max': '30***'},
    '31***-40***': {'min': '31***', 'max': '40***'},
    '41***-50***': {'min': '41***', 'max': '50***'},
    '51***-61***': {'min': '51***', 'max': '60***'}
}
df["Postal Code"] = df["Postal Code"].apply(lambda x: next((key for key, value in rango_codigo_postal.items() if value['min'] <= x <= value['max']), 'NA'))
df.head()

,Name,Age,Email,Phone,Postal Code,Credit Card
0,Estefanía Vendrell-Blanch,medium,benitezedmundo@example.com,+34979 52 98 66,01***-10***,5d9f454ce695dc7953f7423f5d997f8f3b91a6c3b0c096...
1,Jordi del Mascaró,senior,zpaniagua@example.net,+34945 649 215,21***-30***,ead1a4eb7f082eb2e60f523cbcee0dca2a79ce377d4612...
2,Ruy Teruel Fabra,medium,xtrillo@example.com,+34 824 67 56 26,21***-30***,02c972f1f91748f7e6bc95f2572b9bd7a0375bfa26e6d6...
3,Máxima Porcel Pinto,senior,valverdejose-antonio@example.com,+34 946 027 028,11***-20***,403ee98f2e0ed94d63e37d2220925c19bbe0a3c4d48ca4...
4,Jenny del Guillen,medium,soledadferrer@example.net,+34 877 512 059,41***-50***,5ea8829400ca4995fb7d9d951a206633bb0c42a24b5833...


Al campo del correo electrónico le aplicaremos ruido haciendo uso de una de las funciones que definimos la principio del cuaderno.

In [459]:
df["Email"] = df['Email'].apply(generalizeEmail)
df["Email"].head()

0          ben***********@example.com
1               zpa******@example.net
2                 xtr****@example.com
3    val*****************@example.com
4           sol**********@example.net
Name: Email, dtype: object

Por último, en cuanto a columnas como el nombre o el número de teléfono, el nombre se elimina del conjunto de datos debido a que se trata de un identificador directo. Sin embargo, podemos aplicar técnicas como seudónimos. Es importante recalcar que estos seudónimos deben ser únicos y robustos, es decir, no deben ser reversibles por partes no autorizadas a través de la deducción o el cálculo de los valores originales del identificador directo a partir de los seudónimos.

Respecto al número de teléfono, podríamos aplicar técnicas como agregar ruido o utilizar funciones de hash, entre otras. En nuestro caso, hemos decidido deshacernos de estas dos columnas.

In [461]:
df = df[["Age","Email","Postal Code","Credit Card"]]
df.head(100)

,Age,Email,Postal Code,Credit Card
0,medium,ben***********@example.com,01***-10***,5d9f454ce695dc7953f7423f5d997f8f3b91a6c3b0c096...
1,senior,zpa******@example.net,21***-30***,ead1a4eb7f082eb2e60f523cbcee0dca2a79ce377d4612...
2,medium,xtr****@example.com,21***-30***,02c972f1f91748f7e6bc95f2572b9bd7a0375bfa26e6d6...
3,senior,val*****************@example.com,11***-20***,403ee98f2e0ed94d63e37d2220925c19bbe0a3c4d48ca4...
4,medium,sol**********@example.net,41***-50***,5ea8829400ca4995fb7d9d951a206633bb0c42a24b5833...
...,...,...,...,...
95,senior,ube****@example.com,01***-10***,6b7d901df4b785b5f039cc680883db3ffcda5d80843da2...
96,medium,cco****@example.com,41***-50***,f1f71d8775253629b530ca81c106b5b21170cad0d0430e...
97,senior,her************@example.net,11***-20***,4772dc12d42ad41b8fd6ec4b90a595cc9016a60e29ddb5...
98,senior,eus********@example.org,01***-10***,a2adb28de60a248c5ca06a03509a4f43ab52e24e195ed7...


In [462]:
df.to_csv("../Data/Anonimized_dataset.csv", index=False)